In [1]:
import h5py
import numpy as np
import xarray as xr

In [2]:
arrays = {}
f = h5py.File('/scratch/hydrodl/data/raw/cluster_regions/clusterRegions.mat')
for k, v in f.items():
    arrays[k] = np.array(v).T
et = xr.open_dataset('/scratch/hydrodl/data/processed/1d/monthly/et.nc')

In [3]:
dat = arrays['CLregions']
#dat[dat<0] = 0
clusters = xr.Dataset({'regions': xr.DataArray(dat, dims=('lat', 'lon'), coords=(et.lat, et.lon))}).astype(int)
clusters

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360)
Coordinates:
  * lat      (lat) float64 89.5 88.5 87.5 86.5 85.5 ... -86.5 -87.5 -88.5 -89.5
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Data variables:
    regions  (lat, lon) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0

In [4]:
for i, cl in enumerate(['Subtropic', 'TemperateNH', 'Tropic', 'ColdNH', 'SH']):
    clusters[cl] = clusters.regions == i + 1

In [5]:
clusters.attrs.update({'regions': ['Subtropic', 'TemperateNH', 'Tropic', 'ColdNH', 'SH']})

In [6]:
!rm '/scratch/hydrodl/data/processed/1d/static/clusterRegions.nc'

rm: cannot remove '/scratch/hydrodl/data/processed/1d/static/clusterRegions.nc': No such file or directory


In [7]:
clusters.to_netcdf('/scratch/hydrodl/data/processed/1d/static/clusterRegions.nc')

In [8]:
arrays = {}
f = h5py.File('/scratch/hydrodl/data/raw/cluster_regions/lat_lon_904.mat')
for k, v in f.items():
    arrays[k] = np.array(v)[0]
train_pixels = [(np.argwhere(lat == et.lat.values).item(), np.argwhere(lon == et.lon.values).item()) for lat, lon in zip(arrays['lat'], arrays['lon'])]
arrays = {}
f = h5py.File('/scratch/hydrodl/data/raw/cluster_regions/lat_lon_global.mat')
for k, v in f.items():
    arrays[k] = np.array(v)[0]
eval_pixels = [(np.argwhere(lat == et.lat.values).item(), np.argwhere(lon == et.lon.values).item()) for lat, lon in zip(arrays['lat'], arrays['lon'])]

In [9]:
ds = np.zeros((180, 360), dtype=int)
for lat, lon in eval_pixels:
    ds[lat, lon] = 2
for lat, lon in train_pixels:
    ds[lat, lon] = 1

In [10]:
cv_mask = xr.Dataset({'mask': xr.DataArray(ds, dims=('lat', 'lon'), coords=(et.lat, et.lon))})
cv_mask

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360)
Coordinates:
  * lat      (lat) float64 89.5 88.5 87.5 86.5 85.5 ... -86.5 -87.5 -88.5 -89.5
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Data variables:
    mask     (lat, lon) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0

In [11]:
plt.figure(figsize=(18, 8))
(cv_mask.mask).plot()

NameError: name 'plt' is not defined

In [12]:
cv_mask.to_netcdf('/scratch/hydrodl/data/processed/1d/static/cv_mask.nc')